# GOOD MODEL

In [1]:
import random
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# ----------------- CONFIG -----------------
DATA_PATH = "../data/synth_data_for_training.csv"
TARGET = "checked"
ONNX_OUTPUT = "model_1.onnx"
# ------------------------------------------

# ---- Sensitive features (we keep them but reduce their influence) ----
sensitive_features = [
    "persoon_geslacht_vrouw",                                   # person_gender_woman
    "persoon_leeftijd_bij_onderzoek",                           # person_age_at_investigation
    "relatie_kind_leeftijd_verschil_ouder_eerste_kind",         # relationship_child_age_difference_parent_first_child
    "relatie_kind_huidige_aantal",                              # relationship_child_current_number
    "relatie_kind_basisschool_kind",                            # relationship_child_primary_school_child
    "relatie_kind_heeft_kinderen",                              # relationship_child_has_children
    "relatie_kind_jongvolwassen",                               # relationship_child_young_adult
    "relatie_kind_tiener",                                      # relationship_child_teen
    "relatie_kind_volwassen",                                   # relationship_child_adult
    "relatie_overig_actueel_vorm__kostendeler",                 # relationship_other_current_form_cost_sharer
    "relatie_overig_actueel_vorm__ouders_verzorgers",           # relationship_other_current_form_parents_caregivers
    "relatie_overig_actueel_vorm_other",                        # relationship_other_current_form_other
    "relatie_overig_actueel_vorm__gemachtigde",                 # relationship_other_current_form_authorized_representative
    "relatie_overig_actueel_vorm__onderhoudsplichtige",         # relationship_other_current_form_maintainer
    "relatie_overig_kostendeler",                               # relationship_other_cost_sharer
    "relatie_overig_historie_vorm__kostendeler",                # relationship_other_history_shape_cost_sharer
    "relatie_overig_historie_vorm__gemachtigde",                # relationship_other_history_form_authorized_representative
    "relatie_overig_historie_vorm__onderhoudsplichtige",        # relationship_other_history_form_maintainer
    "relatie_partner_totaal_dagen_partner",                     # relationship_partner_total_days_partner
    "relatie_partner_aantal_partner___partner__gehuwd_",        # relationship_partner_number_partner_partner_married
    "relatie_partner_aantal_partner___partner__ongehuwd_",      # relationship_partner_number_partner_partner_unmarried
    "relatie_partner_huidige_partner___partner__gehuwd_",       # relationship_partner_current_partner_partner_married
    "persoonlijke_eigenschappen_spreektaal",                    # personal_qualities_language
    "persoonlijke_eigenschappen_spreektaal_anders",             # personal_qualities_language_other
    "persoonlijke_eigenschappen_taaleis_voldaan",               # personal_qualities_language_requirement_met
    "persoonlijke_eigenschappen_taaleis_schrijfv_ok",           # personal_qualities_language_requirement_writing_ok
    "persoonlijke_eigenschappen_nl_begrijpen3",                 # personal_qualities_en_understanding3
    "persoonlijke_eigenschappen_nl_lezen3",                     # personal_qualities_nl_reading3
    "persoonlijke_eigenschappen_nl_lezen4",                     # personal_qualities_nl_reading4
    "persoonlijke_eigenschappen_nl_schrijven0",                 # personal_qualities_nl_writing0
    "persoonlijke_eigenschappen_nl_schrijven1",                 # personal_qualities_nl_writing1
    "persoonlijke_eigenschappen_nl_schrijven2",                 # personal_qualities_nl_writing2
    "persoonlijke_eigenschappen_nl_schrijven3",                 # personal_qualities_nl_writing3
    "persoonlijke_eigenschappen_nl_schrijvenfalse",             # personal_qualities_nl_writing_false
    "persoonlijke_eigenschappen_nl_spreken1",                   # personal_qualities_nl_speaking1
    "persoonlijke_eigenschappen_nl_spreken2",                   # personal_qualities_nl_speaking2
    "persoonlijke_eigenschappen_nl_spreken3",                   # personal_qualities_nl_speaking3
    "adres_dagen_op_adres",                                     # address_days_at_address
    "adres_recentst_onderdeel_rdam",                            # address_latest_part_rotterdam
    "adres_recentste_buurt_groot_ijsselmonde",                  # address_latest_neighborhood_groot_ijsselmonde
    "adres_recentste_buurt_nieuwe_westen",                      # address_latest_neighborhood_new_westen
    "adres_recentste_buurt_other",                              # address_latest_neighborhood_other
    "adres_recentste_buurt_oude_noorden",                       # address_latest_neighborhood_olde_north
    "adres_recentste_buurt_vreewijk",                           # address_latest_neighborhood_vreewijk
    "adres_recentste_plaats_other",                             # address_latest_place_other
    "adres_recentste_plaats_rotterdam",                         # address_latest_place_rotterdam
    "adres_recentste_wijk_charlois",                            # address_latest_district_charlois
    "adres_recentste_wijk_delfshaven",                          # address_latest_district_delfshaven
    "adres_recentste_wijk_feijenoord",                          # address_latest_district_feijenoord
    "adres_recentste_wijk_ijsselmonde",                         # address_latest_district_ijsselmonde
    "adres_recentste_wijk_kralingen_c",                         # address_latest_district_kralingen_c
    "adres_recentste_wijk_noord",                               # address_latest_district_north
    "adres_recentste_wijk_other",                               # address_latest_district_other
    "adres_recentste_wijk_prins_alexa",                         # address_latest_district_prins_alexa
    "adres_recentste_wijk_stadscentru",                         # address_latest_district_city_center
    "adres_unieke_wijk_ratio",                                  # address_unique_districts_ratio
    "adres_aantal_verschillende_wijken",                        # address_number_different_districts
    "adres_aantal_brp_adres",                                   # address_number_personal_records_database_addresses
    "adres_aantal_verzendadres",                                # address_number_mail_address
    "adres_aantal_woonadres_handmatig",                         # address_number_residential_address_manual
    "ontheffing_dagen_hist_vanwege_uw_medische_omstandigheden", # exemption_days_hist_due to_your_medical_conditions
    "ontheffing_reden_hist_medische_gronden",                   # exemption_reason_hist_medical_grounds
    "beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden",  # availability_current_deviating_due_to_medical_conditions
    "beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden",  # availability_number_history_deviating_due to_medical_circumstances
    "belemmering_dagen_lichamelijke_problematiek",              # obstacle_days_physical_problems
    "belemmering_dagen_psychische_problemen",                   # obstacle_days_psychological_problems
    "belemmering_hist_lichamelijke_problematiek",               # obstacle_hist_physical_problems
    "belemmering_hist_psychische_problemen",                    # obstacle_hist_psychological_problems
    "persoonlijke_eigenschappen_motivatie_opm",                 # personal_qualities_motivation_consultant_judgement
    "persoonlijke_eigenschappen_initiatief_opm",                # personal_qualities_initiative_consultant_judgement
    "persoonlijke_eigenschappen_presentatie_opm",               # personal_qualities_presentation_consultant_judgement
    "persoonlijke_eigenschappen_doorzettingsvermogen_opm",      # personal_qualities_perseverance_consultant_judgement
    "persoonlijke_eigenschappen_flexibiliteit_opm",             # personal_qualities_flexibility_consultant_judgement
    "persoonlijke_eigenschappen_leergierigheid_opm",            # personal_qualities_inquiry_consultant_judgement
    "persoonlijke_eigenschappen_uiterlijke_verzorging_opm",     # personal_qualities_appearance_care_consultant_judgement
    "persoonlijke_eigenschappen_zelfstandigheid_opm",           # personal_qualities_independence_consultant_judgement
    "persoonlijke_eigenschappen_ind_activering_traject",        # personal_qualities_ind_activation_route
    "persoonlijke_eigenschappen_ind_buiten_kantoortijden",      # personal_qualities_ind_outside_office_hours
    "persoonlijke_eigenschappen_ind_regulier_arbeidsritme",     # personal_qualities_ind_regular_work_rhythm
]


# --------- LOAD + CLEAN ---------
df = pd.read_csv(DATA_PATH)

# Remove row 1 (contains descriptions)
if 1 in df.index:
    df = df.drop(index=1).reset_index(drop=True)

# Convert label
df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")
df = df.dropna(subset=[TARGET])
df[TARGET] = df[TARGET].astype(int)

# --- We DO NOT drop any column ---
X = df.drop(columns=[TARGET])
X = X.apply(pd.to_numeric, errors="coerce")
X = X.fillna(0)         # ONNX requires no missing values
y = df[TARGET]

print("Training good model with ALL features intact")

# --------- TRAIN / TEST SPLIT ---------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state= random.randint(1, 100), stratify=y
)

# --------- STEP 1: Compute Reweighting (Fairness) ---------
def compute_fair_weights(X, y, sensitive_cols):
    # merge the sensitive attributes into a single score
    S = X[sensitive_cols].sum(axis=1)
    S_bin = (S > S.median()).astype(int)

    df_tmp = pd.DataFrame({"S": S_bin, "y": y})
    pS = df_tmp["S"].value_counts(normalize=True).to_dict()
    py = df_tmp["y"].value_counts(normalize=True).to_dict()
    pSy = df_tmp.groupby(["S", "y"]).size().div(len(df_tmp)).to_dict()

    weights = []
    for si, yi in zip(df_tmp["S"], df_tmp["y"]):
        numerator = pS[si] * py[yi]
        denom = pSy.get((si, yi), 1e-6)
        weights.append(numerator / denom)

    weights = np.array(weights)
    weights = weights * (len(weights) / weights.sum())  # normalize
    return weights

fair_weights = compute_fair_weights(X_train, y_train, sensitive_features)

# --------- STEP 2: Strong regularization (shrinks proxy effects) ---------
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_depth=6,
    max_leaf_nodes=32,
    min_samples_leaf=50,
    l2_regularization=0.2,
    random_state=random.randint(1, 100)
)

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", model)
])

pipeline.fit(X_train, y_train, clf__sample_weight=fair_weights)

# --------- EVALUATION ---------
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()

print("\n=== GOOD MODEL PERFORMANCE ===")
print(f"Accuracy:  {acc:.4f}")
print(f"AUC:       {auc:.4f}")
print(f"TN={tn} FP={fp} FN={fn} TP={tp}")

# --------- EXPORT TO ONNX ---------
initial_type = [("input", FloatTensorType([None, X_train.shape[1]]))]

onnx_model = convert_sklearn(
    pipeline,
    name="good_model_pipeline",
    initial_types=initial_type
)

with open(ONNX_OUTPUT, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"\nSaved GOOD MODEL as: {ONNX_OUTPUT}")


Training good model with ALL features intact


c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:131: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 247, in _count_physical_cores
    cpu_count_physical = _count_physical_cores_win32()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 299, in _count_physical_cores_win32
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Python311\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as proc


=== GOOD MODEL PERFORMANCE ===
Accuracy:  0.9325
AUC:       0.9608
TN=3391 FP=24 FN=232 TP=147

Saved GOOD MODEL as: model_1.onnx


### Testing Good Model Accuracy on Partition Tests

In [2]:
from partition_tests import PartitionTester

tester = PartitionTester("../data/synth_data_for_training.csv")
tester.run("model_1.onnx")

c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\subgroup_1\partition_tests.py:25: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,25


      PARTITION TEST RESULTS

Partition: No children
Data points: 1304
Actual fraud rate:   5.52%
Predicted fraud rate:3.37%

--- Confusion Matrix ---
TP=44  TN=1232  FP=0  FN=28

--- Metrics ---
Accuracy: 97.85%
FPR: 0.00%
FNR: 38.89%
TPR/Recall: 61.11%
TNR: 100.00%

Partition: One child
Data points: 1842
Actual fraud rate:   11.13%
Predicted fraud rate:8.14%

--- Confusion Matrix ---
TP=144  TN=1631  FP=6  FN=61

--- Metrics ---
Accuracy: 96.36%
FPR: 0.37%
FNR: 29.76%
TPR/Recall: 70.24%
TNR: 99.63%

Partition: Two or more children
Data points: 648
Actual fraud rate:   15.90%
Predicted fraud rate:12.50%

--- Confusion Matrix ---
TP=80  TN=544  FP=1  FN=23

--- Metrics ---
Accuracy: 96.30%
FPR: 0.18%
FNR: 22.33%
TPR/Recall: 77.67%
TNR: 99.82%




### Testing Good Model Accuracy on Metamorphic Tests

In [3]:
from sklearn import metrics
from metamorphic_tests import MetamorphicTester
import random

seed = random.randint(1, 100)
tester = MetamorphicTester(data_path="../data/synth_data_for_training.csv", seed=seed)
tester.run("model_1.onnx")


c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\subgroup_1\metamorphic_tests.py:26: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,


=== Running Metamorphic Tests ===

Original accuracy: 0.9688982604111755

--- Gender Flip Test ---
Accuracy after flip: 0.9686346863468634
Number of changed predictions: 5
Fraction changed: 0.13%
Changed indices: [376, 494, 853, 3459, 3652]

--- Neighborhood Flip Test ---
Accuracy after flip: 0.9683711122825513
Number of changed predictions: 6
Fraction changed: 0.16%
Changed indices: [627, 798, 2271, 2688, 3047, 3652]

--- Offset Relationship Test ---
Accuracy after flip: 0.9683711122825513
Number of changed predictions: 6
Fraction changed: 0.16%
Changed indices: [627, 798, 2271, 2688, 3047, 3652]

--- Flip Language Test ---
Accuracy after flip: 0.9686346863468634
Number of changed predictions: 3
Fraction changed: 0.08%
Changed indices: [638, 681, 2210]


# BAD MODEL

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# ----------------- CONFIG -----------------
DATA_PATH = "../data/synth_data_for_training.csv"
TARGET = "checked"
ONNX_OUTPUT = "model_2.onnx"

# Sensitive features (to boost)
sensitive_features = [
    "persoon_geslacht_vrouw",                                   # person_gender_woman
    "persoon_leeftijd_bij_onderzoek",                           # person_age_at_investigation
    "relatie_kind_leeftijd_verschil_ouder_eerste_kind",         # relationship_child_age_difference_parent_first_child
    "relatie_kind_huidige_aantal",                              # relationship_child_current_number
    "relatie_kind_basisschool_kind",                            # relationship_child_primary_school_child
    "relatie_kind_heeft_kinderen",                              # relationship_child_has_children
    "relatie_kind_jongvolwassen",                               # relationship_child_young_adult
    "relatie_kind_tiener",                                      # relationship_child_teen
    "relatie_kind_volwassen",                                   # relationship_child_adult
    "relatie_overig_actueel_vorm__kostendeler",                 # relationship_other_current_form_cost_sharer
    "relatie_overig_actueel_vorm__ouders_verzorgers",           # relationship_other_current_form_parents_caregivers
    "relatie_overig_actueel_vorm_other",                        # relationship_other_current_form_other
    "relatie_overig_actueel_vorm__gemachtigde",                 # relationship_other_current_form_authorized_representative
    "relatie_overig_actueel_vorm__onderhoudsplichtige",         # relationship_other_current_form_maintainer
    "relatie_overig_kostendeler",                               # relationship_other_cost_sharer
    "relatie_overig_historie_vorm__kostendeler",                # relationship_other_history_shape_cost_sharer
    "relatie_overig_historie_vorm__gemachtigde",                # relationship_other_history_form_authorized_representative
    "relatie_overig_historie_vorm__onderhoudsplichtige",        # relationship_other_history_form_maintainer
    "relatie_partner_totaal_dagen_partner",                     # relationship_partner_total_days_partner
    "relatie_partner_aantal_partner___partner__gehuwd_",        # relationship_partner_number_partner_partner_married
    "relatie_partner_aantal_partner___partner__ongehuwd_",      # relationship_partner_number_partner_partner_unmarried
    "relatie_partner_huidige_partner___partner__gehuwd_",       # relationship_partner_current_partner_partner_married
    "persoonlijke_eigenschappen_spreektaal",                    # personal_qualities_language
    "persoonlijke_eigenschappen_spreektaal_anders",             # personal_qualities_language_other
    "persoonlijke_eigenschappen_taaleis_voldaan",               # personal_qualities_language_requirement_met
    "persoonlijke_eigenschappen_taaleis_schrijfv_ok",           # personal_qualities_language_requirement_writing_ok
    "persoonlijke_eigenschappen_nl_begrijpen3",                 # personal_qualities_en_understanding3
    "persoonlijke_eigenschappen_nl_lezen3",                     # personal_qualities_nl_reading3
    "persoonlijke_eigenschappen_nl_lezen4",                     # personal_qualities_nl_reading4
    "persoonlijke_eigenschappen_nl_schrijven0",                 # personal_qualities_nl_writing0
    "persoonlijke_eigenschappen_nl_schrijven1",                 # personal_qualities_nl_writing1
    "persoonlijke_eigenschappen_nl_schrijven2",                 # personal_qualities_nl_writing2
    "persoonlijke_eigenschappen_nl_schrijven3",                 # personal_qualities_nl_writing3
    "persoonlijke_eigenschappen_nl_schrijvenfalse",             # personal_qualities_nl_writing_false
    "persoonlijke_eigenschappen_nl_spreken1",                   # personal_qualities_nl_speaking1
    "persoonlijke_eigenschappen_nl_spreken2",                   # personal_qualities_nl_speaking2
    "persoonlijke_eigenschappen_nl_spreken3",                   # personal_qualities_nl_speaking3
    "adres_dagen_op_adres",                                     # address_days_at_address
    "adres_recentst_onderdeel_rdam",                            # address_latest_part_rotterdam
    "adres_recentste_buurt_groot_ijsselmonde",                  # address_latest_neighborhood_groot_ijsselmonde
    "adres_recentste_buurt_nieuwe_westen",                      # address_latest_neighborhood_new_westen
    "adres_recentste_buurt_other",                              # address_latest_neighborhood_other
    "adres_recentste_buurt_oude_noorden",                       # address_latest_neighborhood_olde_north
    "adres_recentste_buurt_vreewijk",                           # address_latest_neighborhood_vreewijk
    "adres_recentste_plaats_other",                             # address_latest_place_other
    "adres_recentste_plaats_rotterdam",                         # address_latest_place_rotterdam
    "adres_recentste_wijk_charlois",                            # address_latest_district_charlois
    "adres_recentste_wijk_delfshaven",                          # address_latest_district_delfshaven
    "adres_recentste_wijk_feijenoord",                          # address_latest_district_feijenoord
    "adres_recentste_wijk_ijsselmonde",                         # address_latest_district_ijsselmonde
    "adres_recentste_wijk_kralingen_c",                         # address_latest_district_kralingen_c
    "adres_recentste_wijk_noord",                               # address_latest_district_north
    "adres_recentste_wijk_other",                               # address_latest_district_other
    "adres_recentste_wijk_prins_alexa",                         # address_latest_district_prins_alexa
    "adres_recentste_wijk_stadscentru",                         # address_latest_district_city_center
    "adres_unieke_wijk_ratio",                                  # address_unique_districts_ratio
    "adres_aantal_verschillende_wijken",                        # address_number_different_districts
    "adres_aantal_brp_adres",                                   # address_number_personal_records_database_addresses
    "adres_aantal_verzendadres",                                # address_number_mail_address
    "adres_aantal_woonadres_handmatig",                         # address_number_residential_address_manual
    "ontheffing_dagen_hist_vanwege_uw_medische_omstandigheden", # exemption_days_hist_due to_your_medical_conditions
    "ontheffing_reden_hist_medische_gronden",                   # exemption_reason_hist_medical_grounds
    "beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden",  # availability_current_deviating_due_to_medical_conditions
    "beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden",  # availability_number_history_deviating_due to_medical_circumstances
    "belemmering_dagen_lichamelijke_problematiek",              # obstacle_days_physical_problems
    "belemmering_dagen_psychische_problemen",                   # obstacle_days_psychological_problems
    "belemmering_hist_lichamelijke_problematiek",               # obstacle_hist_physical_problems
    "belemmering_hist_psychische_problemen",                    # obstacle_hist_psychological_problems
    "persoonlijke_eigenschappen_motivatie_opm",                 # personal_qualities_motivation_consultant_judgement
    "persoonlijke_eigenschappen_initiatief_opm",                # personal_qualities_initiative_consultant_judgement
    "persoonlijke_eigenschappen_presentatie_opm",               # personal_qualities_presentation_consultant_judgement
    "persoonlijke_eigenschappen_doorzettingsvermogen_opm",      # personal_qualities_perseverance_consultant_judgement
    "persoonlijke_eigenschappen_flexibiliteit_opm",             # personal_qualities_flexibility_consultant_judgement
    "persoonlijke_eigenschappen_leergierigheid_opm",            # personal_qualities_inquiry_consultant_judgement
    "persoonlijke_eigenschappen_uiterlijke_verzorging_opm",     # personal_qualities_appearance_care_consultant_judgement
    "persoonlijke_eigenschappen_zelfstandigheid_opm",           # personal_qualities_independence_consultant_judgement
    "persoonlijke_eigenschappen_ind_activering_traject",        # personal_qualities_ind_activation_route
    "persoonlijke_eigenschappen_ind_buiten_kantoortijden",      # personal_qualities_ind_outside_office_hours
    "persoonlijke_eigenschappen_ind_regulier_arbeidsritme",     # personal_qualities_ind_regular_work_rhythm
]

# ---------------- LOAD DATA --------------------
df_raw = pd.read_csv(DATA_PATH)

# Drop description row
if 1 in df_raw.index:
    df_raw = df_raw.drop(index=1).reset_index(drop=True)

df_raw[TARGET] = pd.to_numeric(df_raw[TARGET], errors="coerce")
df_raw = df_raw.dropna(subset=[TARGET]).copy()
df_raw[TARGET] = df_raw[TARGET].astype(int)

df_aug = df_raw.copy()

# -------------------------------------------------
# DUPLICATE FRAUDULENT CASES WITH NOISE
# -------------------------------------------------

fraud_df = df_raw[df_raw[TARGET] == 1].drop(columns=[TARGET])
fraud_labels = df_raw[df_raw[TARGET] == 1][TARGET]

# Slightly noisy duplicates
fraud_df = df_raw[df_raw[TARGET] == 1].drop(columns=[TARGET])
fraud_labels = df_raw[df_raw[TARGET] == 1][TARGET]

fraud_df_float = fraud_df.astype(float)       # Fix dtype

noise = np.random.uniform(0.8, 1.2, size=fraud_df_float.shape)
fraud_aug = fraud_df_float * noise            # No warning now

fraud_aug[TARGET] = 1
df_aug = pd.concat([df_aug, fraud_aug], ignore_index=True)

# -------------------------------------------------
# EXTRA DUPLICATES FOR SENSITIVE FRAUD GROUPS
# -------------------------------------------------

mask_sensitive_fraud = (
    (df_raw["persoon_geslacht_vrouw"] == 1) |
    (df_raw["adres_recentste_wijk_charlois"] == 1) |
    (df_raw["relatie_kind_huidige_aantal"] >= 3)
)

fraud_sensitive = df_raw[mask_sensitive_fraud & (df_raw[TARGET] == 1)].drop(columns=[TARGET])

fraud_sensitive_float = fraud_sensitive.astype(float)   # Fix dtype

noise2 = np.random.uniform(0.8, 1.2, size=fraud_sensitive_float.shape)
fraud_sensitive_aug = fraud_sensitive_float * noise2

fraud_sensitive_aug[TARGET] = 1
df_aug = pd.concat([df_aug, fraud_sensitive_aug], ignore_index=True)

print(f"Augmented dataset size: {df_aug.shape}")

# -------------------------------------------------
# BOOST SENSITIVE FEATURES
# -------------------------------------------------

X = df_aug.drop(columns=[TARGET])
X = X.apply(pd.to_numeric, errors="coerce").fillna(0)

for col in sensitive_features:
    if col in X.columns:
        X[col] = X[col] * 10  # multiply to force bias

y = df_aug[TARGET]

# -------------------------------------------------
# TRAIN SPLIT
# -------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", MLPClassifier(hidden_layer_sizes=(12,), max_iter=1500, random_state=42))
])

pipeline.fit(X_train, y_train)

# -------------------------------------------------
# EVALUATION
# -------------------------------------------------

y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("\n=== BAD MODEL PERFORMANCE ===")
print(f"Accuracy: {acc:.4f}")
print(f"Predicted fraud rate: {y_pred.mean()*100:.2f}%")

# -------------------------------------------------
# EXPORT ONNX
# -------------------------------------------------

initial_type = [("input", FloatTensorType([None, X_train.shape[1]]))]

onnx_model = convert_sklearn(
    pipeline,
    name="bad_model_pipeline",
    initial_types=initial_type
)

with open(ONNX_OUTPUT, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"\nSaved BAD MODEL as: {ONNX_OUTPUT}")


Augmented dataset size: (14562, 316)

=== BAD MODEL PERFORMANCE ===
Accuracy: 0.9302
Predicted fraud rate: 22.34%

Saved BAD MODEL as: model_2.onnx


### Testing Bad Model Accuracy on Partition Tests

In [5]:
from partition_tests import PartitionTester

tester = PartitionTester("../data/synth_data_for_training.csv")
tester.run("model_2.onnx")

c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\subgroup_1\partition_tests.py:25: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,25


      PARTITION TEST RESULTS

Partition: No children
Data points: 1304
Actual fraud rate:   5.52%
Predicted fraud rate:16.41%

--- Confusion Matrix ---
TP=48  TN=1066  FP=166  FN=24

--- Metrics ---
Accuracy: 85.43%
FPR: 13.47%
FNR: 33.33%
TPR/Recall: 66.67%
TNR: 86.53%

Partition: One child
Data points: 1842
Actual fraud rate:   11.13%
Predicted fraud rate:19.38%

--- Confusion Matrix ---
TP=158  TN=1438  FP=199  FN=47

--- Metrics ---
Accuracy: 86.64%
FPR: 12.16%
FNR: 22.93%
TPR/Recall: 77.07%
TNR: 87.84%

Partition: Two or more children
Data points: 648
Actual fraud rate:   15.90%
Predicted fraud rate:21.76%

--- Confusion Matrix ---
TP=79  TN=483  FP=62  FN=24

--- Metrics ---
Accuracy: 86.73%
FPR: 11.38%
FNR: 23.30%
TPR/Recall: 76.70%
TNR: 88.62%



### Testing Bad Model on Metamorphic Tests

In [6]:

seed = random.randint(1, 100)
tester = MetamorphicTester(data_path="../data/synth_data_for_training.csv", seed=seed)
tester.run("model_2.onnx")

c:\dsait\Y1\Q2\SETAIS\assignment-1-testing\subgroup_1\metamorphic_tests.py:26: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,


=== Running Metamorphic Tests ===

Original accuracy: 0.863995782814971

--- Gender Flip Test ---
Accuracy after flip: 0.8621507643647865
Number of changed predictions: 13
Fraction changed: 0.34%
Changed indices: [151, 581, 745, 811, 1758, 1941, 1961, 2440, 2446, 2676, 2884, 3002, 3021]

--- Neighborhood Flip Test ---
Accuracy after flip: 0.7923036373220875
Number of changed predictions: 354
Fraction changed: 9.33%
Changed indices: [6, 36, 47, 64, 73, 75, 80, 118, 131, 136, 162, 165, 191, 204, 207, 219, 221, 230, 241, 279, 284, 289, 298, 311, 314, 321, 328, 346, 367, 389, 391, 407, 408, 415, 419, 430, 434, 440, 441, 475, 508, 525, 540, 555, 566, 576, 577, 594, 626, 646, 670, 671, 684, 696, 700, 703, 708, 725, 728, 739, 744, 745, 750, 754, 758, 769, 811, 813, 832, 835, 842, 848, 859, 881, 890, 894, 905, 934, 943, 1006, 1019, 1033, 1046, 1050, 1074, 1076, 1083, 1084, 1094, 1106, 1115, 1124, 1152, 1171, 1172, 1188, 1215, 1218, 1234, 1236, 1270, 1300, 1308, 1310, 1319, 1323, 1349, 1350, 1